In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import pandas as pd

# Kết nối với Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
import torchvision.transforms as transforms
import datasets
import os
from datasets import load_dataset

In [4]:
# Load dataset from folder
dataset = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/splits")

# Define preprocessing
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
def transform(example):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
    ])
    example["pixel_values"] = [transform(img.convert("RGB")) for img in example["image"]]
    return example

dataset = dataset.map(transform, batched=True)
dataset = dataset.remove_columns(["image"]).rename_column("label", "labels")

Resolving data files:   0%|          | 0/240 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
num_labels = len(dataset["train"].features["labels"].names)
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=num_labels)
model.to(device)

training_args = TrainingArguments(
        output_dir="models/outputs",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        save_total_limit=2,
        logging_dir="logs",
        logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"],
    tokenizer=feature_extractor,
)

trainer.train()
model.save_pretrained("models/outputs/vit_finetuned")

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-f1fc7df9e380>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thehaodev96 (thehaodev96-aio-vietnam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.097400,0.059353
2,0.025400,0.023005
3,0.017500,0.016974
4,0.015200,0.014831
5,0.014300,0.014211


In [6]:
def evaluate():
    metrics = trainer.evaluate()
    print("Evaluation metrics:", metrics)
    return metrics

In [13]:
!pip install Pillow

In [14]:
from PIL import Image
def predict(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
    ])
    pixel_values = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(pixel_values)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()

    label_names = dataset["train"].features["labels"].names
    return label_names[predicted_class]

In [15]:
evaluate()

Evaluation metrics: {'eval_loss': 0.014210673049092293, 'eval_runtime': 35.3727, 'eval_samples_per_second': 6.785, 'eval_steps_per_second': 0.848, 'epoch': 5.0}


{'eval_loss': 0.014210673049092293,
 'eval_runtime': 35.3727,
 'eval_samples_per_second': 6.785,
 'eval_steps_per_second': 0.848,
 'epoch': 5.0}

In [16]:
# Predict on test set
def predict_test_set():
    test_dir = "/content/drive/MyDrive/splits/test"
    classes = os.listdir(test_dir)
    results = {}

    for class_name in classes:
        class_path = os.path.join(test_dir, class_name)
        if os.path.isdir(class_path):
            results[class_name] = []
            print(f"Predictions for class: {class_name}")
            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)
                prediction = predict(image_path)
                results[class_name].append((image_name, prediction))
                print(f"Image: {image_name} -> Predicted: {prediction}")
    return results

In [17]:
predict_test_set()

Predictions for class: Nematode
Image: nematode_27.jpg -> Predicted: Nematode
Image: nematode_16.jpg -> Predicted: Nematode
Image: nematode_22.jpg -> Predicted: Nematode
Image: nematode_4.jpg -> Predicted: Nematode
Image: nematode_25.jpg -> Predicted: Nematode
Image: nematode_28.jpg -> Predicted: Nematode
Predictions for class: Pink Disease
Image: pink_disease_17.jpg -> Predicted: Pink Disease
Image: pink_disease_22.jpg -> Predicted: Pink Disease
Image: pink_disease_18.jpg -> Predicted: Pink Disease
Image: pink_disease_1.jpg -> Predicted: Pink Disease
Image: pink_disease_12.jpg -> Predicted: Pink Disease
Image: pink_disease_24.jpg -> Predicted: Pink Disease


{'Nematode': [('nematode_27.jpg', 'Nematode'),
  ('nematode_16.jpg', 'Nematode'),
  ('nematode_22.jpg', 'Nematode'),
  ('nematode_4.jpg', 'Nematode'),
  ('nematode_25.jpg', 'Nematode'),
  ('nematode_28.jpg', 'Nematode')],
 'Pink Disease': [('pink_disease_17.jpg', 'Pink Disease'),
  ('pink_disease_22.jpg', 'Pink Disease'),
  ('pink_disease_18.jpg', 'Pink Disease'),
  ('pink_disease_1.jpg', 'Pink Disease'),
  ('pink_disease_12.jpg', 'Pink Disease'),
  ('pink_disease_24.jpg', 'Pink Disease')]}